# GWP # 1

s101 MScFE 560 Financial Markets 

Jaepil Choi

## My Portfolio: Portfolio A

Buy 1 stock and short 1 stock

In [166]:
from typing import Tuple

import pandas as pd
import numpy as np

from itertools import permutations

## Step 1

For each investment, you are given statistics in this .csv file the stock’s.
You also have the correlation and covariance matrix of historical returns (download this
.csv file)

1. Average return
2. Volatility
3. Skewness
4. Kurtosis

You must compute the portfolio’s average return and portfolio volatility. You do not
need to compute the portfolio skewness and portfolio kurtosis.

In [167]:
statistics_df = pd.read_csv('MScFE560_FM_GWP1_Data.csv', nrows=4, index_col=0)
statistics_df = statistics_df.iloc[:, :5]
statistics_df

,AAPL,AMZN,NFLX,META,GOOG
Mean,0.044858,0.010362,-0.029675,-0.006934,0.029035
St Dev,0.009320,0.009557,0.013409,0.011582,0.008376
Skew,-0.335173,-0.312865,-3.101654,-1.936506,-0.197700
Kurt,5.034093,4.729806,41.968015,23.289236,4.310171


In covariance matrix, `GOOGL` (Alphabet Inc Class A) is given instead of `GOOG` (Alphabet Inc Class C). 

To match the name, I changed `GOOGL` to `GOOG`

In [168]:
corr_matrix_df = pd.read_csv('MScFE560_FM_GWP1_Data.csv', skiprows=5, nrows=6, header=1, index_col=0)
corr_matrix_df = corr_matrix_df.iloc[:, :5]
corr_matrix_df.rename({'GOOGL': 'GOOG'}, axis=0, inplace=True)
corr_matrix_df.rename({'GOOGL': 'GOOG'}, axis=1, inplace=True)
corr_matrix_df

,AAPL,AMZN,NFLX,META,GOOG
AAPL,1.000000,0.660739,0.460041,0.594785,0.698431
AMZN,0.660739,1.000000,0.593812,0.626198,0.679323
NFLX,0.460041,0.593812,1.000000,0.515892,0.492697
META,0.594785,0.626198,0.515892,1.000000,0.668024
GOOG,0.698431,0.679323,0.492697,0.668024,1.000000


### 1-1 Portfolio's average return

In [169]:
mean = statistics_df.loc['Mean', :] 
std = statistics_df.loc['St Dev', :]

In [170]:
sid_list = statistics_df.columns
sid_list

Index(['AAPL', 'AMZN', 'NFLX', 'META', 'GOOG'], dtype='object')

Since I can long 1 stock and short 1 stock, there are $ 2^5 $ number of cases

In [171]:
longshort_cases = permutations(sid_list, 2)
longshort_cases = list(longshort_cases)

Portfolio return formula:

$$ PortfolioReturn = w_a * r_a + w_b * r_b $$

What is "weight"?

Weight is the position betted on the stock. 

What does it mean to have negative weight? 

It means you're shorting. 

By shorting, you borrow stocks and sell them to the market: which gives you cash. 

You can use this cash to long the stock. 

As you can see, long-short can have varing book size. 

In [172]:
def get_portfolio_avg_return(case: tuple) -> Tuple[tuple, float]:
    long_sid, short_sid = case
    
    long_avg_return = mean.loc[long_sid]
    short_avg_return = -mean.loc[short_sid]

    returns = np.array([long_avg_return, short_avg_return])
    weights = np.array([0.5, 0.5])

    portfolio_avg_returns = np.dot(weights, returns)

    return (case, portfolio_avg_returns)

In [173]:
longshort_portfolio_returns = []

for case in longshort_cases:
    longshort_portfolio_returns.append(get_portfolio_avg_return(case))

In [174]:
longshort_portfolio_returns = sorted(longshort_portfolio_returns, key=lambda x: x[1], reverse=True)

In [175]:
# Top 5 return portfolios
longshort_portfolio_returns[:5]

[(('AAPL', 'NFLX'), 0.0372663865),
 (('GOOG', 'NFLX'), 0.029354981500000002),
 (('AAPL', 'META'), 0.025896114499999998),
 (('AMZN', 'NFLX'), 0.020018404),
 (('GOOG', 'META'), 0.0179847095)]

In [176]:
# Bottom 5 return portfolios
longshort_portfolio_returns[-5:]

[(('META', 'GOOG'), -0.0179847095),
 (('NFLX', 'AMZN'), -0.020018404),
 (('META', 'AAPL'), -0.025896114499999998),
 (('NFLX', 'GOOG'), -0.029354981500000002),
 (('NFLX', 'AAPL'), -0.0372663865)]

### 1-2 Portfolio's volatility

$$ PortfolioVariance = w_a^2 \sigma_a^2 + w_b^2 \sigma_b^2 + 2 \rho w_a w_b \sigma_a \sigma_b $$ 

In [177]:
def get_portfolio_variance(case: tuple) -> Tuple[tuple, float]:
    long_sid, short_sid = case
    
    long_std = std.loc[long_sid]
    short_std = std.loc[short_sid]

    corr_coef = corr_matrix_df.loc[long_sid, short_sid]

    stds = np.array([long_std, short_std])
    weights = np.array([0.5, 0.5])

    portfolio_variance = np.dot(weights ** 2, stds ** 2) + 2 * corr_coef * np.prod(weights) * np.prod(stds)

    return (case, portfolio_variance)

In [178]:
longshort_portfolio_vars = []

for case in longshort_cases:
    longshort_portfolio_vars.append(get_portfolio_variance(case))

In [179]:
longshort_portfolio_vars = sorted(longshort_portfolio_vars, key=lambda x: x[1], reverse=True)

In [180]:
# Top 5 variance portfolios
longshort_portfolio_vars[:5]

[(('NFLX', 'META'), 0.00011854272900667238),
 (('META', 'NFLX'), 0.00011854272900667238),
 (('AMZN', 'NFLX'), 0.00010582904280622674),
 (('NFLX', 'AMZN'), 0.00010582904280622674),
 (('AAPL', 'NFLX'), 9.541393892085575e-05)]

In [181]:
# Botoom 5 variance portfolios
longshort_portfolio_vars[-5:]

[(('AMZN', 'AAPL'), 7.397774388261782e-05),
 (('AMZN', 'GOOG'), 6.755958745209989e-05),
 (('GOOG', 'AMZN'), 6.755958745209989e-05),
 (('AAPL', 'GOOG'), 6.651794872231761e-05),
 (('GOOG', 'AAPL'), 6.651794872231761e-05)]

### Sharpe ratio

In [182]:
port_returns_dict = dict(longshort_portfolio_returns)
port_vars_dict = dict(longshort_portfolio_vars)

In [183]:
longshort_portfolio_sharpe = []

for case in longshort_cases:
    ret = port_returns_dict[case]
    std = np.sqrt(port_vars_dict[case])

    sharpe = ret / std

    longshort_portfolio_sharpe.append((case, sharpe))

In [184]:
longshort_portfolio_sharpe = sorted(longshort_portfolio_sharpe, key=lambda x: x[1], reverse=True)

In [185]:
# Top 5 variance portfolios
longshort_portfolio_sharpe[:5]

[(('AAPL', 'NFLX'), 3.8151478355174087),
 (('GOOG', 'NFLX'), 3.0916307973593753),
 (('AAPL', 'META'), 2.770699465117833),
 (('AAPL', 'AMZN'), 2.0053388225881084),
 (('GOOG', 'META'), 1.9684571979620828)]

### Answer

I'll choose `('AAPL', 'NFLX')` since it has the highest sharpe ratio

In [186]:
portfolio = ('AAPL', 'NFLX')

print(f'Portfolio: Long 1 {portfolio[0]} / Short 1 {portfolio[1]}')
print(f'Portfolio return: {port_returns_dict[portfolio]:.4f}')
print(f'Portfolio std: {np.sqrt(port_vars_dict[portfolio]):.4f}')

Portfolio: Long 1 AAPL / Short 1 NFLX
Portfolio return: 0.0373
Portfolio std: 0.0098


In [187]:
port_vars_dict[portfolio]

9.541393892085575e-05

## Step 2

Answer the following questions:

Each Team Member/Portfolio Manager will answer the following questions about their
own portfolio:

### 2-1 Shorting

#### a. Can this portfolio be shorted? (Hint: Yes, but be sure to explain part b)

## Step 3